<a href="https://colab.research.google.com/github/laribar/TechChallenge2/blob/main/TechChallenge_2_Rev01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [502]:
# Análise de dados e manipulação
import pandas as pd
import numpy as np

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# Manipulação de datas
from datetime import datetime
from pandas.tseries.offsets import BDay

# Verificação de versões (opcional)
import sklearn
print(f"Scikit-learn version: {sklearn.__version__}")

Scikit-learn version: 1.6.1


In [503]:
# Parte 2 - Criar Features

# Criar Features
for i in range(1, 6):
    df[f'Lag{i}'] = df['Close'].shift(i)

df['Daily_Return'] = df['Close'].pct_change()
df['MA_7'] = df['Close'].rolling(window=7).mean()
df['MA_30'] = df['Close'].rolling(window=30).mean()
df['Volatility'] = df['Close'].pct_change().rolling(window=7).std()
df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))

df['BB_Mid'] = df['Close'].rolling(window=20).mean()
std_dev = df['Close'].rolling(window=20).std().squeeze()
df['BB_Upper'] = df['BB_Mid'] + (std_dev * 2)
df['BB_Lower'] = df['BB_Mid'] - (std_dev * 2)

delta = df['Close'].diff(1)
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

# Armazena a variável alvo antes do dropna()
y = df['Close'].copy()

# Remover NaNs
df.dropna(inplace=True)

# Criar X e y mantendo os índices alinhados
X = df.drop(columns=['Close'])
y = y.loc[X.index]  # Mantém y alinhado com X

# Separação treino e teste baseada no tempo
train_size = int(len(df) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

In [504]:
# Parte 3 - Teste e Ajuste para Estacionariedade

from statsmodels.tsa.stattools import adfuller

def test_stationarity(series, threshold=0.05):
    result = adfuller(series)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    return result[1] <= threshold

def make_stationary(y_train, y_test, X_train, X_test):
    if not test_stationarity(y_train):
        y_train = y_train.diff().dropna()
        y_test = y_test.diff().dropna()
        print("Série não era estacionária. Aplicado diferenciação.")

        # Ajustando X para manter alinhamento com y
        X_train = X_train.loc[y_train.index]
        X_test = X_test.loc[y_test.index]

    return y_train, y_test, X_train, X_test

# Aplicar ajuste de estacionariedade
y_train, y_test, X_train, X_test = make_stationary(y_train, y_test, X_train, X_test)

ADF Statistic: -0.4151031575720165
p-value: 0.907546386321177
Série não era estacionária. Aplicado diferenciação.


In [505]:
# Parte 4 - Normalização para Modelos Lineares (evitando data leakage)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit apenas no treino
X_test_scaled = scaler.transform(X_test)  # Transform no teste

In [508]:
# Parte 5 - Ajustando Modelos

# Random Forest NÃO precisa de escalonamento, então usamos os dados brutos
best_params_rf = {'n_estimators': 210, 'max_depth': 9}
rf_model = RandomForestRegressor(**best_params_rf)
rf_model.fit(X_train, y_train)  # Treinamos com os dados originais, sem escalonamento

# Previsões
y_pred_rf = rf_model.predict(X_test)  # Também sem escalonamento

# Se aplicamos diferenciação, precisamos reverter os valores previstos
if 'diff' in str(y_train.name).lower():  # Verifica se aplicamos diff()
    y_pred_rf = pd.Series(y_pred_rf, index=y_test.index).cumsum() + y.iloc[train_size]

# Avaliação do Modelo
mse = mean_squared_error(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)
r2 = r2_score(y_test, y_pred_rf)

print(f'MSE: {mse:.2f}')
print(f'MAE: {mae:.2f}')
print(f'R²: {r2:.4f}')


MSE: 224877.96
MAE: 218.72
R²: 0.9060
